# NARX


In [1]:
import numpy as np

from lib.read_data import cycles, u_desvio, y, y_desvio, y_ref

model_name = "NARX"


In [2]:
import hickle as hkl


def gerar_NARX(y, u):
    y_prev = y[:-1, :]
    u_prev = u[:-1, :]
    u_now = u[1:, :]

    # Termos lineares
    X_lin = np.hstack([y_prev, u_prev, u_now])
    # Termos (não-lineares)
    X_quad = X_lin**2
    X_quartic = X_lin**4

    # Matriz de regressão final
    X = np.hstack([X_lin, X_quad, X_quartic])

    # Saídas futuras
    Y = y[1:, :]

    # Estimando parâmetros via mínimos quadrados
    theta = np.linalg.lstsq(X, Y)[0]
    return theta


def inferir_NARX(theta, y, u):
    N = len(y)
    y_narx = np.zeros_like(y)
    y_narx[0] = y[0]  # inicializa com o primeiro valor real

    for k in range(1, N):
        y_old = y_narx[k - 1]
        u_old = u[k - 1]
        u_now = u[k]

        phi_lin = np.concatenate([y_old, u_old, u_now])
        phi_quad = phi_lin**2
        phi_quartic = phi_lin**4

        phi = np.concatenate([phi_lin, phi_quad, phi_quartic])

        y_narx[k] = phi @ theta
    return y_narx


theta = gerar_NARX(y_desvio, u_desvio)
hkl.dump(theta, f"../export/{model_name}.hkl")
print("Parâmetros estimados:\n", theta.shape)

# from lib.read_data import u_ref
# u_sp = np.array([610, 220, 135, 105]) - u_ref
# u_desvio = np.full((200, 4), u_sp)

y_narx = inferir_NARX(theta, y_desvio, u_desvio)
y_narx = y_narx + y_ref  # Convertendo o valor desvio em variavel de engenharia


Parâmetros estimados:
 (33, 3)


In [3]:
# Plotando resultados
from lib.plot import plot_comparison

plot_comparison(cycles, y, y_narx, model_name)
